In [1]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import autograd.numpy as np

import copy

# c. local modules
import CCP as estimate
from contracept_model_3 import child_model3
import back as solver


In [2]:
model = child_model3()

In [9]:
V, pnc = solver.BackwardsInduction3(model)
dataset = model.sim_data(pnc)

In [14]:
choice = dataset.d
covariates = dataset.x


In [17]:
import numpy as np
from scipy.optimize import minimize
# Function to estimate CCP using Hotz and Miller method
def estimate_ccp(dta, num_choices, num_periods, discount_factor):
    num_obs = dta.shape[0]
    covariates = dta['x'].values.reshape((num_obs, num_periods))

    # Initialize CCP matrix
    ccp_matrix = np.zeros((num_choices, num_periods))

    for t in range(num_periods):
        # Get data and covariates for time period t
        choices_t = dta['d'].values
        covariates_t = covariates[:, t]

        # Estimate CCP for each choice
        for i in range(num_choices):
            # Filter data for choice i
            choice_i_data = choices_t == i
            choice_i_covariates = covariates_t[choice_i_data]

            # Estimate CCP for choice i using Hotz and Miller method
            ccp_estimate = estimate_choice_prob(choice_i_covariates, covariates_t, discount_factor)

            # Update CCP matrix
            ccp_matrix[i, t] = ccp_estimate

    return ccp_matrix
# Function to estimate choice probabilities using Hotz and Miller method
def estimate_choice_prob(choice_data, covariates, discount_factor):
    num_obs = choice_data.shape[0]
    num_covariates = covariates.shape[1]

    # Estimate parameters using maximum likelihood
    initial_guess = np.zeros(num_covariates)
    result = minimize(negative_log_likelihood, initial_guess, args=(choice_data, covariates, discount_factor))
    params = result.x

    # Calculate choice probabilities
    choice_probs = np.exp(covariates @ params) / np.sum(np.exp(covariates @ params))

    return choice_probs

# Function to calculate negative log-likelihood
def negative_log_likelihood(params, choice_data, covariates, discount_factor):
    utilities = covariates @ params
    log_likelihood = np.sum(np.log(choice_data @ np.exp(utilities)))
    discount_factor_adjustment = (1 - discount_factor) * np.sum(utilities)
    return -log_likelihood + discount_factor_adjustment



dta = model.read_data()

num_choices = 2  # Specify the number of choices (e.g., 2 for binary choice)
num_periods = model.T  # Specify the number of periods (e.g., 1 for cross-sectional data)
discount_factor = 0.95  # Specify the discount factor

ccp_matrix = estimate_ccp(dta, num_choices, num_periods, discount_factor)
print("Estimated CCP Matrix:")
print(ccp_matrix)

ValueError: cannot reshape array of size 20982 into shape (20982,27)

In [ ]:

estimate_ccp(data, covariates, 2,model.T, model.beta)